In [132]:
import numpy as np
from scipy.stats import rankdata

# Clusterized ranking

![Logo](https://harzing.com/img/i/inclrank.jpeg)

In [133]:
M = np.array([
    [5, 3, 1, 2, 8, 4, 6, 7],
    [5, 4, 3, 1, 8, 2, 6, 7],
    [1, 7, 5, 4, 8, 2, 3, 6],
    [6, 4, 2.5, 2.5, 8, 1, 7, 5],
    [8, 2, 4, 6, 3, 5, 1, 7],
    [5, 6, 4, 3, 2, 1, 7, 8],
    [6, 1, 2, 3, 5, 4, 8, 7],
    [5, 1, 3, 2, 7, 4, 6, 8],
    [6, 1, 3, 2, 5, 4, 7, 8],
    [5, 3, 2, 1, 8, 4, 6, 7],
    [7, 1, 3, 2, 6, 4, 5, 8],
    [1, 6, 5, 3, 8, 4, 2, 7]
])
n, m = M.shape

Here is how we find **average** ranking.

In [134]:
average_rank = rankdata(np.average(M, axis=0))
average_rank

array([ 5. ,  3.5,  2. ,  1. ,  7. ,  3.5,  6. ,  8. ])

And this way we can get **median** ranking.

In [135]:
median_rank = rankdata(np.median(M, axis=0))
median_rank

array([ 5. ,  2.5,  2.5,  1. ,  8. ,  4. ,  6. ,  7. ])

Next we need to compute **kernel of disagreement**.

In [136]:
adj = np.zeros((m, m), dtype=np.bool)
kernel = []
for i in range(m):
    for j in range(i + 1, m):
        if (average_rank[i] - average_rank[j])*(median_rank[i] - median_rank[j]) < 0:
            kernel.append([i, j])
            adj[i][j] = True
            adj[j][i] = True
kernel

[[4, 7]]

Now that we have a graph of the disagreement, we can easily find a full component via Depth First Search.

In [137]:
def dfs(i, used):
    if i in used:
        return []
    used.add(i)
    
    res = [i]
    for j in range(m):
        if adj[i][j]:
            res += dfs(j, used)
    return res

Last thing to do, is to iterate in the correct order, and don't forget to print a whole cluster when needed.

In [138]:
order = sorted(range(m), key=lambda i: (average_rank[i], median_rank[i]))
order

[3, 2, 1, 5, 0, 6, 4, 7]

In [139]:
result = []
used = set()
for i in order:
    cluster = dfs(i, used)
    if len(cluster) > 0:
        result.append(cluster)
result

[[3], [2], [1], [5], [0], [6], [4, 7]]